In [1]:
import os
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split


In [4]:
items_df = pd.read_csv("../data/cleaned_data/fashion_dataset.csv")
print("Total items:", len(items_df))
items_df.head()


Total items: 289222


,image_name,x_1,y_1,x_2,y_2,width,height,area,category_id,category_name,category_type,category_type_name,eval_status,positive_attributes,absolute_path,num_attributes
0,img/Sheer_Pleated-Front_Blouse/img_00000001.jpg,72,79,232,273,160,194,31040,3,Blouse,1,upper-body,train,"[717, 818]",../data\img/Sheer_Pleated-Front_Blouse/img_000...,2
1,img/Sheer_Pleated-Front_Blouse/img_00000002.jpg,67,59,155,161,88,102,8976,3,Blouse,1,upper-body,train,"[717, 818]",../data\img/Sheer_Pleated-Front_Blouse/img_000...,2
2,img/Sheer_Pleated-Front_Blouse/img_00000003.jpg,65,65,156,200,91,135,12285,3,Blouse,1,upper-body,val,"[141, 717, 837, 956]",../data\img/Sheer_Pleated-Front_Blouse/img_000...,4
3,img/Sheer_Pleated-Front_Blouse/img_00000004.jpg,51,62,167,182,116,120,13920,3,Blouse,1,upper-body,train,[716],../data\img/Sheer_Pleated-Front_Blouse/img_000...,1
4,img/Sheer_Pleated-Front_Blouse/img_00000005.jpg,46,88,166,262,120,174,20880,3,Blouse,1,upper-body,test,"[349, 405, 717, 810]",../data\img/Sheer_Pleated-Front_Blouse/img_000...,4


In [5]:
num_users = 500
num_items = len(items_df)
rng = np.random.default_rng(42)

rows = []
for u in range(num_users):
    # pick 20 distinct items per user
    picks = rng.choice(num_items, size=20, replace=False)
    for i in picks:
        rating = rng.integers(1, 6)       # ratings in [1,5]
        rows.append((u, i, rating))

inter_df = pd.DataFrame(rows, columns=["user_idx","item_idx","rating"])
print("Synthetic interactions:", inter_df.shape)
inter_df.head()


Synthetic interactions: (10000, 3)


,user_idx,item_idx,rating
0,0,223831,4
1,0,207506,1
2,0,58267,4
3,0,282163,4
4,0,242872,2


In [6]:
train_val, test_df = train_test_split(inter_df, test_size=0.2, random_state=42)
train_df, val_df   = train_test_split(train_val, test_size=0.25, random_state=42)

print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))


Train: 6000 Val: 2000 Test: 2000
